# 실습: 3-에이전트 논문 추천 시스템

## 목표
이 실습에서는 3개의 에이전트가 협력하는 논문 추천 시스템을 만듭니다:
1. **검색 에이전트**: 사용자 질문과 관련된 논문 5개 검색
2. **요약 에이전트**: 검색된 논문들을 분석하고 요약
3. **추천 에이전트**: 분석 결과를 바탕으로 최적의 논문 3개 추천

## TODO 개수: 3개
- TODO 1: 3개의 도구 만들기
- TODO 2: 3개의 에이전트 함수 만들기
- TODO 3: LangGraph 워크플로우 구성

**예상 소요 시간: 약 1시간**

## 준비 단계: 라이브러리 설치 (이미 완료)

In [ ]:
!pip install -q qdrant-client transformers torch accelerate sentence-transformers pandas langchain langgraph

## 준비 단계: 라이브러리 임포트 (이미 완료)

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Any, Optional
import operator
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

## 준비 단계: 데이터 로드 및 전처리 (이미 완료)

In [ ]:
df = pd.read_csv('arxiv_ai.csv')
df = df[['title', 'summary', 'published', 'authors']].dropna()
df = df.sample(500, random_state=42).reset_index(drop=True)
df['published_year'] = pd.to_datetime(df['published']).dt.year
df['text'] = df['title'] + ' ' + df['summary']

print(f"✅ 전처리 완료: {len(df)}개 논문")

## 준비 단계: 임베딩 모델 로드 (이미 완료)

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✅ 임베딩 모델 로드 완료")

## 준비 단계: Qdrant 설정 및 데이터 저장 (이미 완료)

In [ ]:
client = QdrantClient(":memory:")
collection_name = "arxiv_papers"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

points = []
for idx, row in df.iterrows():
    vector = embedding_model.encode(row['text']).tolist()
    point = PointStruct(
        id=idx,
        vector=vector,
        payload={
            "title": row['title'],
            "summary": row['summary'],
            "year": int(row['published_year']),
            "authors": row['authors']
        }
    )
    points.append(point)
    if (idx + 1) % 100 == 0:
        print(f"{idx + 1}개 논문 처리 완료")

client.upsert(collection_name=collection_name, points=points)
print(f"\n✅ {len(points)}개 논문이 벡터 DB에 저장됨")

## 준비 단계: Granite 모델 로드 (이미 완료)

In [ ]:
model_path = "ibm-granite/granite-4.0-micro"
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
llm_model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device if device == "cuda" else None)
llm_model.eval()

def generate_response(prompt: str, max_tokens: int = 300) -> str:
    chat = [{"role": "user", "content": prompt}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_tokens = tokenizer(chat_text, return_tensors="pt").to(device)
    output = llm_model.generate(**input_tokens, max_new_tokens=max_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=False)
    assistant_response = response.split("<|start_of_role|>assistant<|end_of_role|>")[-1]
    assistant_response = assistant_response.split("<|end_of_text|>")[0].strip()
    return assistant_response

print(f"✅ Granite 모델 로드 완료 ({device})")

---
# 실습 시작
---

## TODO 1: 3개의 도구(Tools) 만들기 ⭐

에이전트가 사용할 3개의 도구를 만드세요.

In [ ]:
# TODO 1-1 (Korean): search_papers_tool을 구현하세요
# TODO 1-1 (English): Implement search_papers_tool
# HINT: @tool 데코레이터를 사용하세요
# HINT: def search_papers_tool(query: str, top_k: int = 5) -> str:
# HINT: client.query_points()로 검색하고 .points로 결과를 받으세요
# HINT: 각 논문의 제목, 연도, 유사도, 요약(150자)을 포함하여 반환하세요



# TODO 1-2 (Korean): summarize_papers_tool을 구현하세요
# TODO 1-2 (English): Implement summarize_papers_tool
# HINT: def summarize_papers_tool(papers_info: str) -> str:
# HINT: generate_response()를 사용하여 논문들을 분석하세요
# HINT: 프롬프트에 "핵심 기술", "적용 가능성", "연관성"을 분석하도록 요청하세요



# TODO 1-3 (Korean): recommend_papers_tool을 구현하세요
# TODO 1-3 (English): Implement recommend_papers_tool
# HINT: def recommend_papers_tool(summary: str, papers_info: str) -> str:
# HINT: summary와 papers_info를 모두 받아서 최고의 논문 3개를 추천하세요
# HINT: 추천 이유, 활용 방법, 읽는 순서를 포함하도록 프롬프트를 작성하세요



tools = [search_papers_tool, summarize_papers_tool, recommend_papers_tool]
print("✅ TODO 1 완료: 3개 도구 생성")

## TODO 2: 3개의 에이전트 함수 만들기 ⭐

각 에이전트는 이전 에이전트의 결과를 받아서 처리합니다.

In [ ]:
class AgentState(TypedDict):
    query: str
    search_results: str
    summary: str
    recommendations: str
    next_step: str

# TODO 2-1 (Korean): search_agent 함수를 구현하세요
# TODO 2-1 (English): Implement search_agent function
# HINT: def search_agent(state: AgentState) -> AgentState:
# HINT: state["query"]를 사용하여 search_papers_tool을 호출하세요
# HINT: 결과를 state["search_results"]에 저장하세요
# HINT: state["next_step"] = "summarize"로 설정하세요



# TODO 2-2 (Korean): summarize_agent 함수를 구현하세요
# TODO 2-2 (English): Implement summarize_agent function
# HINT: def summarize_agent(state: AgentState) -> AgentState:
# HINT: state["search_results"]를 사용하여 summarize_papers_tool을 호출하세요
# HINT: 결과를 state["summary"]에 저장하세요
# HINT: state["next_step"] = "recommend"로 설정하세요



# TODO 2-3 (Korean): recommend_agent 함수를 구현하세요
# TODO 2-3 (English): Implement recommend_agent function
# HINT: def recommend_agent(state: AgentState) -> AgentState:
# HINT: state["summary"]와 state["search_results"]를 모두 recommend_papers_tool에 전달하세요
# HINT: 결과를 state["recommendations"]에 저장하세요
# HINT: state["next_step"] = "end"로 설정하세요



def router(state: AgentState) -> str:
    return state["next_step"]

print("✅ TODO 2 완료: 3개 에이전트 생성")

## TODO 3: LangGraph 워크플로우 구성 ⭐

3개 에이전트를 연결하는 워크플로우를 만드세요.

In [ ]:
# TODO 3 (Korean): LangGraph 워크플로우를 구성하세요
# TODO 3 (English): Configure LangGraph workflow
# HINT: workflow = StateGraph(AgentState)
# HINT: workflow.add_node()로 3개 에이전트 추가 ("search", "summarize", "recommend")
# HINT: workflow.set_entry_point("search")로 시작점 설정
# HINT: workflow.add_conditional_edges()로 조건부 엣지 추가
#       - "search" → router → {"summarize": "summarize", "end": END}
#       - "summarize" → router → {"recommend": "recommend", "end": END}
# HINT: workflow.add_edge("recommend", END)로 마지막 엣지 추가
# HINT: app = workflow.compile()



print("✅ TODO 3 완료: 워크플로우 구성 완료")

---
# 테스트
---

## 시스템 실행 함수

In [ ]:
def run_recommendation_system(user_query: str):
    initial_state = {
        "query": user_query,
        "search_results": "",
        "summary": "",
        "recommendations": "",
        "next_step": "search"
    }
    
    print("=" * 70)
    print(f"📝 사용자 질문: {user_query}")
    print("=" * 70)
    
    result = app.invoke(initial_state)
    
    print("\n" + "=" * 70)
    print("🔍 [1단계: 검색 결과]")
    print("=" * 70)
    print(result["search_results"])
    
    print("\n" + "=" * 70)
    print("📊 [2단계: 논문 분석]")
    print("=" * 70)
    print(result["summary"])
    
    print("\n" + "=" * 70)
    print("⭐ [3단계: 최종 추천 Top 3]")
    print("=" * 70)
    print(result["recommendations"])
    
    return result

## 테스트 1: 컴퓨터 비전

In [ ]:
result = run_recommendation_system(
    "I need papers about deep learning for computer vision in autonomous driving"
)

## 테스트 2: 자연어 처리

In [ ]:
result = run_recommendation_system(
    "Natural language processing for chatbot development with transformers"
)

## 결론

축하합니다! 3-에이전트 논문 추천 시스템을 완성했습니다.

### 학습한 내용:
1. ✅ LangChain 도구 만들기 (@tool)
2. ✅ 상태 기반 에이전트 구현
3. ✅ LangGraph 워크플로우 구성
4. ✅ 에이전트 간 데이터 전달

### 시스템 흐름:
```
사용자 질문 
    ↓
검색 에이전트 (5개 논문 검색)
    ↓
요약 에이전트 (논문 분석)
    ↓
추천 에이전트 (Top 3 추천)
    ↓
최종 결과
```